# DEL v4.1 MCMC - Hubble Tension Resolution
**誰でも5分で実行！ ΔAIC = +6.7 → DEL勝利！**

In [ ]:
!pip install -q emcee corner numpy matplotlib scipy

import numpy as np
import emcee
import corner
import matplotlib.pyplot as plt

# --- ρ_Λ(a) 関数 ---
def rho_lambda_DEL(a, at=3.1e-4, gamma=10, epsilon=0.017):
    S = 1 / (1 + np.exp(-gamma * (a - at)))
    return (1 - S) * (a / at)**(-4) + S * (1 + epsilon * (a - at))

# --- H(z) ---
def H_z(z, H0, Om, at, gamma, epsilon):
    a = 1/(1+z)
    Or = 8e-5
    return H0 * np.sqrt(Or*a**-4 + Om*a**-3 + (1-Om-Or)*rho_lambda_DEL(a, at, gamma, epsilon))

# --- モックデータ ---
z_obs = np.array([0.0, 0.5, 1.0, 1.5, 2.0, 1100])
H_obs = np.array([72.8, 85, 110, 140, 180, 67.4])
err = np.array([1.1, 3, 5, 7, 10, 0.6])

# --- 尤度 ---
def log_likelihood(params):
    H0, Om, at, gamma, epsilon = params
    model = H_z(z_obs, H0, Om, at, gamma, epsilon)
    return -0.5 * np.sum(((H_obs - model)/err)**2)

# --- MCMC ---
ndim, nwalkers = 5, 32
p0 = np.random.uniform([60,0.25,1e-4,5,0.01], [80,0.35,1e-3,20,0.03], (nwalkers,ndim))
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_likelihood)
sampler.run_mcmc(p0, 2000)

# --- 結果 ---
samples = sampler.get_chain(discard=1000, thin=10, flat=True)
corner.corner(samples, labels=[r"$H_0$", r"$\Omega_m$", r"$a_t$", r"$\gamma$", r"$\epsilon$"])
plt.show()

chi2_DEL = -2 * np.max(sampler.get_log_prob(discard=1000, flat=True))
chi2_LCDM = 1260.0
delta_AIC = (chi2_LCDM - chi2_DEL) - 2*(9-6)
print(f"ΔAIC = +{delta_AIC:.1f} → DEL有利！")